In [1]:
using ReinforcementLearning, Intervals, Flux

const RLBase = ReinforcementLearningBase

#Need to make it so I use include("some.jl") instead but for now
#using NBInclude
#@nbinclude("RIS_Solvers.ipynb")

println("Done RL Import")

Base.@kwdef mutable struct MyGrid <: AbstractEnv
    N::Int
    #We start with squares start spread across the line at co-ordinates 1,2,3,4,5,6,7
    obs::Vector{Float32}
    max_steps::Int    
    
    rewards::Float32
    done::Bool 
    t::Int
end

function MyGrid(; n)
    println("here?")
    img = zeros(Float32, n,n)
    vect = ones(Float32, 3)
    obs = vcat(vect,vec(img))
    println(obs)
    
	MyGrid(n, obs, 10, 0, false, 0)
end

RLBase.action_space(env::MyGrid) = Base.OneTo(25)

function (env::MyGrid)(action) #This function takes the action, makes the environment step, and gives reward
    
    env.obs[action[1]] = 6
    
    env.rewards+=1
    
    env.t+=1
    #Max steps check
    if env.t >= env.max_steps
        env.done = true
    end
end

RLBase.state(env::MyGrid, ::Observation{Vector{Float32}}) = env.obs
RLBase.state_space(env::MyGrid, ::Observation{Vector{Float32}}) = Space(fill( Interval{Int64}(0, (10*env.N) ) , (env.N^2+3)))

RLBase.is_terminated(env::MyGrid) = env.done

function RLBase.reset!(env::MyGrid)
    #Reset Counter and rewards (and termination variable)
    env.t = 0
    env.rewards = 0
    env.done = false    
    
    img = zeros(env.N,env.N)
    vect = ones(3)
    env.obs = vcat(vect,vec(img))
end;

RLBase.reward(env::MyGrid) = env.rewards

RLBase.NumAgentStyle(::MyGrid) = SINGLE_AGENT
RLBase.DynamicStyle(::MyGrid) = SEQUENTIAL
RLBase.ActionStyle(::MyGrid) = MINIMAL_ACTION_SET
RLBase.InformationStyle(::MyGrid) = PERFECT_INFORMATION

RLBase.StateStyle(::MyGrid) = Observation{Vector{Float32}}()

RLBase.RewardStyle(::MyGrid) = STEP_REWARD
RLBase.UtilityStyle(::MyGrid) = GENERAL_SUM
RLBase.ChanceStyle(::MyGrid) = DETERMINISTIC

println("Are we getting this far?")
N=5
env = MyGrid(; n=N)

Done RL Import
Are we getting this far?
here?
Float32[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


# MyGrid

## Traits

| Trait Type        |                          Value |
|:----------------- | ------------------------------:|
| NumAgentStyle     |                  SingleAgent() |
| DynamicStyle      |                   Sequential() |
| InformationStyle  |           PerfectInformation() |
| ChanceStyle       |                Deterministic() |
| RewardStyle       |                   StepReward() |
| UtilityStyle      |                   GeneralSum() |
| ActionStyle       |             MinimalActionSet() |
| StateStyle        | Observation{Vector{Float32}}() |
| DefaultStateStyle | Observation{Vector{Float32}}() |

## Is Environment Terminated?

No

## State Space

`Space{Vector{Interval{Int64, Closed, Closed}}}(Interval{Int64, Closed, Closed}[Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50), Interval{Int64, Closed, Closed}(0, 50)])`

## Action Space

`Base.OneTo(25)`

## Current State

```
Float32[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```


In [2]:
println("asdfghjkl;")

asdfghjkl;


In [3]:
length(state_space(env)) == length(env.obs)

true

In [4]:
UPDATE_FREQ = 32
N_ENV = 1

agent = Agent(
        policy = PPOPolicy(
            approximator = ActorCritic(
                    actor = Chain(
                                x -> ( reshape(x[4:28], (5,5,1,1)), x[1:3]),
                                Parallel(vcat, 
                                        Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                                        Flux.flatten),

                                        Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu)) ),
                                Dense(10, 20, relu),
                                Dense(20,5,relu),
                                Dense(5,3)),

                    critic = Chain(
                                x -> ( reshape(x[4:28], (5,5,1,1)), x[1:3]),
                                Parallel(vcat, 
                                        Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                                        Flux.flatten),

                                        Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu)) ),
                                Dense(10, 20, relu),
                                Dense(20,5,relu),
                                Dense(5,1)),
            
                optimizer = ADAM(1e-3),
            ),
            γ = 0.99f0,
            λ = 0.95f0,
            clip_range = 0.1f0,
            max_grad_norm = 0.5f0,
            n_epochs = 4,
            n_microbatches = 4,
            actor_loss_weight = 1.0f0,
            critic_loss_weight = 0.5f0,
            entropy_loss_weight = 0.001f0,
            update_freq = UPDATE_FREQ,
        ),
        trajectory = PPOTrajectory(;
            capacity = UPDATE_FREQ, 
            state = Matrix{Float32} => (28, N_ENV),
            action = Vector{Int} => (N_ENV,),
            action_log_prob = Vector{Float32} => (N_ENV,),
            reward = Vector{Float32} => (N_ENV,),  
            terminal = Vector{Bool} => (N_ENV,),
        ),
    )


#stop_condition = StopAfterStep(30000, is_show_progress=true)
hook = TotalBatchRewardPerEpisode(N_ENV)

TotalBatchRewardPerEpisode([Float64[]], [0.0], true)

In [5]:
run(agent, env, StopAfterEpisode(8), hook)

Progress:  25%|██████████▎                              |  ETA: 0:00:51

LoadError: BoundsError: attempt to access 1-element view(::Matrix{Float32}, 1, :) with eltype Float32 at index [33]

In [77]:
reshape(env.obs[4:28], (5,5,1,1))

5×5×1×1 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [58]:
env.obs[3]

1.0

In [24]:
piccy = vcat(ones(66),vec(ones(257, 257, 1, 1)))

input_size = length(piccy)
input_2_size = 66
img_size = 257
intermediate_size = 66+484
N=8

actor = Chain(
                                    x -> (reshape(x[input_2_size+1:input_size], (img_size, img_size, 1, 1)) , vec(x[1:input_2_size])), #Reshape to tuple of img and vector
                                    Parallel(vcat, 
                                            Chain(
                                                Conv((5,5), 1 => 1, relu; stride = 1, pad = 2),
                                                MaxPool((3, 3), stride = 3, pad = 1 ),
                                                Conv((3,3), 1 => 1, relu; stride = 1, pad = 1),
                                                MaxPool((2, 2), stride = 2, pad = 0 ), 
                                                Conv((3,3), 1 => 1, relu; stride = 1, pad = 1),
                                                MaxPool((2, 2), stride = 2, pad = 1 ),
                                                Flux.flatten #484
                                            ),

                                            Chain(
                                            Dense(input_2_size, 2*input_2_size, relu),
                                            Dense(2*input_2_size, 4*input_2_size, relu),
                                            Dense(4*input_2_size, 2*input_2_size, relu),
                                            Dense(2*input_2_size, input_2_size, relu),
                                            ) 
                                    
                                    ),
                                    Dense(intermediate_size, 2*intermediate_size, relu),
                                    Dense(2*intermediate_size, 4*intermediate_size,relu),
                                    Dense(4*intermediate_size, 6*intermediate_size,relu),
                                    Dense(6*intermediate_size, 3*intermediate_size,relu),
                                    Dense(3*intermediate_size, 16*N,relu),
                                    Dense(16*N,8*N)
                                )

actor(piccy)

64×1 Matrix{Float64}:
 -0.00015945607284860615
  0.008135492712931815
 -0.002972822429694884
  0.011602450918730974
  0.009035569954637819
  0.008901277931304056
 -0.017736585586259214
  0.005293512118842351
  0.004367446678254119
  0.010146741064251652
  0.0105508340287752
 -0.012956716921473923
  0.01754678678531644
  ⋮
 -0.02487806022444549
  0.010982694687708433
 -0.007708456594378983
 -0.0382446638852149
  0.007793545079391705
 -0.00018703643346538144
  0.01865648728784077
 -0.0034607597684451954
 -0.0015735904782137773
  0.027374764478404257
  0.009506182249492488
 -0.010790780824208323

In [26]:
A = [1 2 3 ; 4 5 6 ; 7 8 9]
B = vec(A)

A == reshape(B, (3,3))



true

NN Testing

In [37]:
food = env.obs

cow = Chain(
                                x -> (x[1:3], reshape(x[4:length(x)], (5,5,1,1)) ),
                                Parallel(vcat, 
                                        Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                                        Flux.flatten),

                                        Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu)) ),
                                Dense(10, 20, relu),
                                Dense(20,5,relu),
                                Dense(5,1))

cow(food)

LoadError: DimensionMismatch("Rank of x and w must match! (1 vs. 4)")

In [42]:
picman = Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                                        Flux.flatten)

mank = ones(5,5,1,1)

#picman(mank)

picman(reshape(env.obs[4:length(env.obs)], (5,5,1,1)))

9×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [43]:
flatty = Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu))

flatty(env.obs[1:3])

1-element Vector{Float64}:
 1.0026763877153413

In [49]:
parallely = Chain(
                                x -> (reshape(x[4:length(x)], (5,5,1,1)), x[1:3] ),
                                Parallel(vcat, 
                                        Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                                        Flux.flatten),

                                        Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu)) )
    )
#reshape(env.obs[4:length(env.obs)], (5,5))
parallely(env.obs)

10×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [3]:
model = Chain(
        Parallel(vcat, 
                Chain(Conv((3,3), 1 => 1, relu; stride = 1, pad = 0),
                Flux.flatten),

                Chain(Dense(3, 6, relu), Dense(6, 3, relu), Dense(3, 1, relu)) ),
        Dense(10, 20, relu),
        Dense(20,5,relu),
        Dense(5,1))

Chain(
  Parallel(
    vcat,
    Chain(
      Conv((3, 3), 1 => 1, relu),       # 10 parameters
      Flux.flatten,
    ),
    Chain(
      Dense(3, 6, relu),                # 24 parameters
      Dense(6, 3, relu),                # 21 parameters
      Dense(3, 1, relu),                # 4 parameters
    ),
  ),
  Dense(10, 20, relu),                  # 220 parameters
  Dense(20, 5, relu),                   # 105 parameters
  Dense(5, 1),                          # 6 parameters
)                   # Total: 14 arrays, 390 parameters, 3.156 KiB.

In [4]:
model(env.obs)

LoadError: DimensionMismatch("Rank of x and w must match! (3 vs. 4)")